# SQL query from table names - Continued

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [5]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [6]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [7]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [8]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [9]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

I see you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [10]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you.


In [11]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

I see you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [12]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you further.


In [ ]:
# --- Zero-Shot Prompt ---
prompt_zero = """
You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables
{tables}

### Question
{question}
"""

# Define your table definitions as a string
text_tables = """
CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);
"""

my_question = "Which users have placed more than 5 orders in 2025?"
print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=my_question))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=my_question), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
Which users have placed more than 5 orders in 2025?


→ Zero-Shot SQL:
 SELECT u.id, u.name
FROM users u
JOIN (
    SELECT user_id, COUNT(*) as num_orders
    FROM orders
    WHERE order_date >= '2025-01-01' AND order_date <= '2025-12-31'
    GROUP BY user_id
    HAVING COUNT(*) > 5
) o ON u.id = o.user_id;


In [ ]:
# --- Single-Domain Few-Shot Prompt  ---
prompt_few_single = """
You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_id, u.name
FROM users u
JOIN orders o ON u.user_id = o.user_id
WHERE o.order_date BETWEEN '2025-03-01' AND '2025-03-31'
GROUP BY u.user_id, u.name
HAVING SUM(o.total_amount) > 1000;

---

### Now You
Tables:
{tables}

Question: {question}
SQL:
"""

q2 = "Which users have placed more than 3 orders?"
print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=q2))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=q2), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


=== Single-Domain Few-Shot Prompt ===

You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_id, u.name
FROM users u
JOIN orders o ON u.user_id = o.user_id
WHERE o.order_date BETWEEN '2025-03-01' AND '2025-03-31'
GROUP BY u.user_id, u.name
HAVING SUM(o.total_amount) > 1000;

---

### Now You
Tables:

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


Question: Which users have placed more than 3 orders?
SQL:


→ Single-Domain SQL:
 SELECT u.id, u.name
FROM users u
JOIN (
    SELECT user_id, COUNT(*) as num_orders
    FROM orders
    GROUP BY user_id
    HAVING COUNT(*) > 3
) o ON u.id = o.user_id;


In [ ]:
# --- Cross-Domain Few-Shot Prompt (two different domains) ---
prompt_few_cross = """
You’re an AI that writes SQL for any domain. Follow these two examples:

### Example 1 (e-commerce)
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)
Question: Total revenue by user in Q2 2025.
SQL:
SELECT u.user_id, SUM(o.total_amount) AS revenue_q2
FROM users u
JOIN orders o ON u.user_id = o.user_id
WHERE o.order_date BETWEEN '2025-04-01' AND '2025-06-30'
GROUP BY u.user_id;

### Example 2 (HR)
Tables:
employees(emp_id, dept_id, salary), departments(dept_id, dept_name)
Question: Average salary per department.
SQL:
SELECT d.dept_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
GROUP BY d.dept_name;

---

### Now You
Tables:
{tables}

Question: {question}
SQL:
"""

q3 = "How many orders were placed each day in July 2025?"
print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=q3))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=q3), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)


=== Cross-Domain Few-Shot Prompt ===

You’re an AI that writes SQL for any domain. Follow these two examples:

### Example 1 (e-commerce)
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)
Question: Total revenue by user in Q2 2025.
SQL:
SELECT u.user_id, SUM(o.total_amount) AS revenue_q2
FROM users u
JOIN orders o ON u.user_id = o.user_id
WHERE o.order_date BETWEEN '2025-04-01' AND '2025-06-30'
GROUP BY u.user_id;

### Example 2 (HR)
Tables:
employees(emp_id, dept_id, salary), departments(dept_id, dept_name)
Question: Average salary per department.
SQL:
SELECT d.dept_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
GROUP BY d.dept_name;

---

### Now You
Tables:

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


Question: How many orders were placed each day in July 2025?
SQL:


→ Cross-Domain SQL:
 SELECT order_date, COU

In [20]:

my_question = "Which users have placed more than 5 orders in 2025?"
q2 = "Which users have placed more than 3 orders?"
q3 = "How many orders were placed each day in July 2025?"
q4 = "Show daily user growth rate month-over-month for 2025."
query = q4

print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=query))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=query), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=query))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=query), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=query), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)

=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
Show daily user growth rate month-over-month for 2025.


→ Zero-Shot SQL:
 ```sql
WITH monthly_user_counts AS (
    SELECT EXTRACT(MONTH FROM order_date) AS month,
           COUNT(DISTINCT user_id) AS user_count
    FROM orders
    WHERE EXTRACT(YEAR FROM order_date) = 2025
    GROUP BY EXTRACT(MONTH FROM order_date)
),
monthly_growth_rate AS (
    SELECT month,
           user_count,
           LAG(user_count) OVER (ORDER BY month) AS prev_user_count,
           (user_count - LAG(user_count) OVER (ORDER BY month)) / LAG(user_count) OVER (ORDER BY month) AS growth_rate
    FROM monthly_user_counts
)
SELEC

In [21]:

my_question = "Which users have placed more than 5 orders in 2025?"
q2 = "Which users have placed more than 3 orders?"
q3 = "How many orders were placed each day in July 2025?"
q4 = "Show daily user growth rate month-over-month for 2025."
q5 = "List users with zero orders but last login within 30 days"
q6 = "Rank products by average order quantity in 2025"
q7 = "Find the 5th highest spender among users."
q8 = "In HR schema, list departments without employees."


In [22]:
query = q5

print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=query))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=query), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=query))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=query), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=query), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)

=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
List users with zero orders but last login within 30 days


→ Zero-Shot SQL:
 SELECT u.id, u.name
FROM users u
LEFT JOIN orders o ON u.id = o.user_id
WHERE o.id IS NULL
AND DATEDIFF(CURDATE(), u.last_login) <= 30;
=== Single-Domain Few-Shot Prompt ===

You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_id, u.name
FROM users u
JOIN orders o ON u.user_id = o.user_id
WHERE 

In [23]:
query = q6
print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=query))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=query), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=query))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=query), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=query), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)

=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
Rank products by average order quantity in 2025


→ Zero-Shot SQL:
 SELECT product_id, AVG(quantity) AS avg_order_quantity
FROM order_details
WHERE order_id IN (
    SELECT id
    FROM orders
    WHERE YEAR(order_date) = 2025
)
GROUP BY product_id
ORDER BY avg_order_quantity DESC;
=== Single-Domain Few-Shot Prompt ===

You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_i

In [24]:
query = q7
print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=query))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=query), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=query))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=query), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=query), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)

=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
Find the 5th highest spender among users.


→ Zero-Shot SQL:
 ```sql
SELECT u.id, u.name, SUM(o.id) AS total_spent
FROM users u
JOIN orders o ON u.id = o.user_id
GROUP BY u.id, u.name
ORDER BY total_spent DESC
LIMIT 1 OFFSET 4;
```
=== Single-Domain Few-Shot Prompt ===

You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_id, u.name
FROM users u
JOIN orders o ON u.user_id 

In [25]:
query = q8
print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=query))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=query), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=query))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=query), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=query), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)

=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
In HR schema, list departments without employees.


→ Zero-Shot SQL:
 SELECT d.department_id, d.department_name
FROM departments d
LEFT JOIN employees e ON d.department_id = e.department_id
WHERE e.employee_id IS NULL;
=== Single-Domain Few-Shot Prompt ===

You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_id, u.name
FROM users u
JOIN orders o ON u.user_id = o.user_id
W

In [28]:
prompt_unified = """
You are an expert SQL generator.  Your job is to produce a correct SQL query *strictly* from the supplied table definitions.  
If you need a table or column not in the schema, respond with an error (see Step 0).

Step 0: List all tables and their columns exactly as given below.  
Step 1: Identify which of those tables are required.  
Step 2: Determine join conditions based *only* on keys present.  
Step 3: Choose fields, filters, grouping, and ordering to answer the question.  
Step 4: Write the final SQL query, using only the given tables/columns and standard SQL.

**Hard constraints** (enforced!):
- Do not invent or reference any table or column not listed below.  
- If the question cannot be answered with the given schema, output  
  `Error: insufficient schema - missing [list of needed tables/columns]`  
  and nothing else.  
- Output only the final SQL (or the exact Error line), no extra commentary.

### Schema
{tables}

### Question
{question}

Your answer should look like:

1. Step 0: …  
2. Step 1: …  
3. Step 2: …  
4. Step 3: …  
5. Step 4: …  
"""

In [30]:
query = q6
print("=== Zero-Shot Prompt ===")
print(prompt_zero.format(tables=text_tables, question=query))

sql_zero = return_CCRMSQL(prompt_zero.format(tables=text_tables, question=query), context=[])
print("\n→ Zero-Shot SQL:\n", sql_zero)


print("=== Single-Domain Few-Shot Prompt ===")
print(prompt_few_single.format(tables=text_tables, question=query))

sql_single = return_CCRMSQL(prompt_few_single.format(tables=text_tables, question=query), context=[])
print("\n→ Single-Domain SQL:\n", sql_single)


print("=== Cross-Domain Few-Shot Prompt ===")
print(prompt_few_cross.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_few_cross.format(tables=text_tables, question=query), context=[])
print("\n→ Cross-Domain SQL:\n", sql_cross)

print("=== Addressing concerns in a Unified Prompt ===")
print(prompt_unified.format(tables=text_tables, question=query))

sql_cross = return_CCRMSQL(prompt_unified.format(tables=text_tables, question=query), context=[])
print("\n→ Unified Prompt SQL:\n", sql_cross)

=== Zero-Shot Prompt ===

You are an AI that writes precise SQL queries given only table definitions.
Generate a single SQL statement that answers the user’s question.
Return _only_ the SQL (no explanation or commentary).

### Tables

CREATE TABLE users (
	id INT PRIMARY KEY,
	name VARCHAR
);

CREATE TABLE orders (
	id INT PRIMARY KEY,
	user_id INT,
	order_date DATE
);


### Question
Rank products by average order quantity in 2025


→ Zero-Shot SQL:
 SELECT product_id, AVG(quantity) AS avg_order_quantity
FROM order_details
WHERE order_id IN (
    SELECT id
    FROM orders
    WHERE YEAR(order_date) = 2025
)
GROUP BY product_id
ORDER BY avg_order_quantity DESC;
=== Single-Domain Few-Shot Prompt ===

You are an SQL expert. Translate each user question into a correct SQL query.
Follow this example pattern exactly:

### Example
Tables:
users(user_id, name), orders(order_id, user_id, total_amount, order_date)

Question: List all users who spent over $1000 in March 2025.
SQL:
SELECT u.user_i

Zero-Shot is good on simple filters	but doesnt use JOIN or uses wrong tables(even hallucinates a table such as "order_details" that is not existing). Single-Domain Few-Shot has very high accuracy for questions that look a lot like the examples—but for questions outside those narrow examples, the model “over-fits” and can’t adapt, like using products and product_name which were not defined in prompt., Cross-Domain Few-Shot hallucinates(Invented tables/columns such as products and order_details from both domain) in unfamiliar schemas despite the attempt in broadening the coverage. It defaulted to your “JOIN X ON X.id = Y.X_id” pattern across all prompts, regardless of whether those tables were defined or relevant. It mixes up HR terms(departments and employees iso users/orders) because example used HR

A prompt that will explicitly address all the limitations from the older prompts will provide us a more controlled output.

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?